### Miscellaneous Concepts

Refer to [/examples/time_series/generate_data.ipynb](https://github.com/serhatsoyer/py4ML/blob/main/examples/time_series/generate_data.ipynb) for the artificial time-series dataset (used in this example) generation  

Previous example: [/examples/time_series/cases/17.ipynb](https://github.com/serhatsoyer/py4ML/blob/main/examples/time_series/cases/17.ipynb)  
Next example: [/examples/toy_datasets.ipynb](https://github.com/serhatsoyer/py4ML/blob/main/examples/toy_datasets.ipynb)

In [1]:
import sys
sys.path.insert(0, '../../') # To be able to reach 'datasets' folder
from pathlib import Path
import numpy as np
from enum import Enum
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, classification_report, confusion_matrix
from keras.utils import np_utils
from keras import backend as ker # For custom loss and custom metric
from keras.layers import Input, Conv1D, MaxPooling1D, concatenate, BatchNormalization, LSTM, Bidirectional, Dropout, Dense
from keras.models import Model

In [2]:
dataset_path = Path.cwd().parent.parent / 'datasets' / 'time_series'

input = np.load(dataset_path / 'input.npy')
output = np.load(dataset_path / 'output.npy')

def print_dataset(X_train, X_test, y_train, y_test, note):
    print(note)
    print(f"{'X_train shape and type:':<25}{X_train.shape} and {X_train.dtype}")
    print(f"{'X_test shape and type:':<25}{X_test.shape} and {X_test.dtype}")
    print(f"{'y_train shape and type:':<25}{y_train.shape} and {y_train.dtype}")
    print(f"{'y_test shape and type:':<25}{y_test.shape} and {y_test.dtype}")


# For regression:
# Use vers=0, within=1, inter=0
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(\
    input[0], \
    output[1][0][0], \
    test_size=0.20, shuffle=True)

print_dataset(X_train_reg, X_test_reg, y_train_reg, y_test_reg, 'Regression:')

# For binary classification:
# Try to classify vers for within=1, inter=0
X_train_bin, X_test_bin, y_train_bin, y_test_bin = train_test_split(\
    np.concatenate((input[0], input[1])), \
    np.concatenate((np.zeros_like(output[1][0][0]), np.ones_like(output[1][0][1]))), \
    test_size=0.20, shuffle=True)

print_dataset(X_train_bin, X_test_bin, y_train_bin, y_test_bin, '\n\nBinary Classification:')

# For multi-class classification:
# Try to classify output region for vers=0, within=1, inter=0
# Partition output into 4 regions
X_train_multi, X_test_multi, y_train_temp_1, y_test_temp_1 = train_test_split(\
    input[0], \
    output[1][0][0], \
    test_size=0.20, shuffle=True)

del input, output # I delete a variable in Python for the first time

# Train data must never be processed with info obtained from test data
percentiles = np.percentile(y_train_temp_1, [25, 50, 75])

y_train_temp_2 = np.zeros_like(y_train_temp_1)
for idx in range(len(y_train_temp_2)):
    if y_train_temp_1[idx] < percentiles[0]:
        y_train_temp_2[idx] = 0
    elif y_train_temp_1[idx] < percentiles[1]:
        y_train_temp_2[idx] = 1
    elif y_train_temp_1[idx] < percentiles[2]:
        y_train_temp_2[idx] = 2
    else:
        y_train_temp_2[idx] = 3

y_train_multi = np_utils.to_categorical(y_train_temp_2)

del y_train_temp_1, y_train_temp_2 # Not used anymore

y_test_temp_2 = np.zeros_like(y_test_temp_1)
for idx in range(len(y_test_temp_2)):
    if y_test_temp_1[idx] < percentiles[0]:
        y_test_temp_2[idx] = 0
    elif y_test_temp_1[idx] < percentiles[1]:
        y_test_temp_2[idx] = 1
    elif y_test_temp_1[idx] < percentiles[2]:
        y_test_temp_2[idx] = 2
    else:
        y_test_temp_2[idx] = 3

y_test_multi = np_utils.to_categorical(y_test_temp_2)

del y_test_temp_1, y_test_temp_2 # Not used anymore

print_dataset(X_train_multi, X_test_multi, y_train_multi, y_test_multi, '\n\nMulti-Class Classification:')
print(f'\nPercentiles: {percentiles}')

Regression:
X_train shape and type:  (24576, 32, 3) and float32
X_test shape and type:   (6144, 32, 3) and float32
y_train shape and type:  (24576,) and float32
y_test shape and type:   (6144,) and float32


Binary Classification:
X_train shape and type:  (49152, 32, 3) and float32
X_test shape and type:   (12288, 32, 3) and float32
y_train shape and type:  (49152,) and float32
y_test shape and type:   (12288,) and float32


Multi-Class Classification:
X_train shape and type:  (24576, 32, 3) and float32
X_test shape and type:   (6144, 32, 3) and float32
y_train shape and type:  (24576, 4) and float32
y_test shape and type:   (6144, 4) and float32

Percentiles: [-8.80615234 -6.60819888 -3.91394669]


In [3]:
num_of_epochs = 8
batch_size = 128

# Proper enumeration in Python:
class NetworkType(Enum):
    Regression = 1
    Binary = 2
    Multi = 3


# Just for demonstration:
def custom_loss(y_true, y_pred):
    return ker.mean(ker.square(y_true - y_pred), axis=1)


# Just for demonstration:
def custom_metric(y_true, y_pred):
    return ker.sum(ker.abs(y_pred - y_true), axis=-1)


def get_model(network_type, X_train):
    """
    Functional API in Keras used this time for illustration
    Instead of Sequential Model in Keras
    """
    conv_filter_size = 32
    conv_kernel_size = 8
    conv_stride = 1
    pool_size = 2
    lstm_size = 32
    drop_prob = 1 / 8
    dense_size = 32
    input = Input((X_train.shape[1], X_train.shape[2]), name='in')
    batchnorm = BatchNormalization(name='bn')(input)
    conv1 = Conv1D(conv_filter_size, conv_kernel_size, strides=conv_stride, activation='relu', name=f'conv1')(batchnorm)
    pool1 = MaxPooling1D(pool_size, name=f'pool1')(conv1)
    conv2 = Conv1D(conv_filter_size, conv_kernel_size, strides=conv_stride, activation='selu', name=f'conv2')(batchnorm)
    pool2 = MaxPooling1D(pool_size, name=f'pool2')(conv2)
    lstm1 = LSTM(lstm_size, return_sequences=True, name='lstm1')(concatenate([pool1, pool2]))
    # If another LSTM follows, return_sequences should be True
    lstm2 = Bidirectional(LSTM(lstm_size, name='lstm2'), name='bidir')(lstm1)
    drop = Dropout(drop_prob, name='drop')(lstm2)
    dense1 = Dense(dense_size, activation='relu', name='dense1')(drop)
    if network_type == NetworkType.Regression:
        out = Dense(1, name='out')(dense1) # Regression
    elif network_type == NetworkType.Binary:
        out = Dense(1, activation='sigmoid', name='out')(dense1) # Binary classification
    elif network_type == NetworkType.Multi:
        out = Dense(4, activation='softmax', name='out')(dense1) # Mult-class classification

    model = Model(inputs=input, outputs=out, name=f'model_{network_type}')
    model.summary()
    if network_type == NetworkType.Regression:
        model.compile(loss=custom_loss, optimizer='adam', metrics=[custom_metric])
    elif network_type == NetworkType.Binary:
        model.compile(loss='binary_crossentropy', optimizer='adam')
    elif network_type == NetworkType.Multi:
        model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model


# Way of analysing intermediate signals:
def get_bn_stats(model, X_train):
    print(f"{'BatchNormalization input shape:':<35}\
        {Model(inputs=model.input, outputs=model.get_layer('bn').input)(X_train).shape}")
    print(f"{'BatchNormalization output shape:':<35}\
        {Model(inputs=model.input, outputs=model.get_layer('bn').output)(X_train).shape}")
    print(f"{'BatchNormalization input mean:':<35}\
        {np.mean(Model(inputs=model.input, outputs=model.get_layer('bn').input)(X_train))}")
    print(f"{'BatchNormalization output mean:':<35}\
        {np.mean(Model(inputs=model.input, outputs=model.get_layer('bn').output)(X_train))}")
    print(f"{'BatchNormalization input std:':<35}\
        {np.std(Model(inputs=model.input, outputs=model.get_layer('bn').input)(X_train))}")
    print(f"{'BatchNormalization output std:':<35}\
        {np.std(Model(inputs=model.input, outputs=model.get_layer('bn').output)(X_train))}")

In [4]:
model_reg = get_model(NetworkType.Regression, X_train_reg)
get_bn_stats(model_reg, X_train_reg)
y_train_reg_pred_before = np.squeeze(model_reg.predict(X_train_reg, verbose=0))
rmse_train_reg_before = np.sqrt(mean_squared_error(y_train_reg, y_train_reg_pred_before))
y_test_reg_pred_before = np.squeeze(model_reg.predict(X_test_reg, verbose=0))
rmse_test_reg_before = np.sqrt(mean_squared_error(y_test_reg, y_test_reg_pred_before))
print('\nBefore Training:')
print(f"{'RMSE Train:':<15}{rmse_train_reg_before:3.2f}")
print(f"{'RMSE Test:':<15}{rmse_test_reg_before:3.2f}")
model_reg.fit(X_train_reg, y_train_reg, batch_size=batch_size, epochs=num_of_epochs, verbose=0)
get_bn_stats(model_reg, X_train_reg)
y_train_reg_pred_after = np.squeeze(model_reg.predict(X_train_reg, verbose=0))
rmse_train_reg_after = np.sqrt(mean_squared_error(y_train_reg, y_train_reg_pred_after))
y_test_reg_pred_after = np.squeeze(model_reg.predict(X_test_reg, verbose=0))
rmse_test_reg_after = np.sqrt(mean_squared_error(y_test_reg, y_test_reg_pred_after))
print('\nAfter Training:')
print(f"{'RMSE Train:':<15}{rmse_train_reg_after:3.2f}")
print(f"{'RMSE Test:':<15}{rmse_test_reg_after:3.2f}")

Metal device set to: Apple M2

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-12-22 15:06:06.248656: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-22 15:06:06.248738: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model_NetworkType.Regression"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 in (InputLayer)                [(None, 32, 3)]      0           []                               
                                                                                                  
 bn (BatchNormalization)        (None, 32, 3)        12          ['in[0][0]']                     
                                                                                                  
 conv1 (Conv1D)                 (None, 25, 32)       800         ['bn[0][0]']                     
                                                                                                  
 conv2 (Conv1D)                 (None, 25, 32)       800         ['bn[0][0]']                     
                                                                       

2022-12-22 15:06:06.801401: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-12-22 15:06:07.128174: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-22 15:06:07.238855: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-22 15:06:07.311810: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-22 15:06:07.311833: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.



Before Training:
RMSE Train:    7.14
RMSE Test:     7.20


2022-12-22 15:06:18.748404: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-22 15:06:19.185874: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-22 15:06:19.230835: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-22 15:06:19.230853: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-22 15:06:19.421432: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-22 15:06:19.441094: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-22 15:06:19.714083: I tensorflow/core/grappler/optimizers/cust

BatchNormalization input shape:            (24576, 32, 3)
BatchNormalization output shape:           (24576, 32, 3)
BatchNormalization input mean:             -0.01970834657549858
BatchNormalization output mean:            0.030349083244800568
BatchNormalization input std:              6.517927169799805
BatchNormalization output std:             1.0719249248504639

After Training:
RMSE Train:    1.20
RMSE Test:     1.21


- Batch normalization layer obtained almost unit std and zero mean as expected only after training
- RMSE reduced significantly both for training and test sets
- Conv and pool layer sizes might be adjusted (this form is just for illustration)

In [5]:
model_bin = get_model(NetworkType.Binary, X_train_bin)
y_train_bin_pred_before = np.float32(model_bin.predict(X_train_bin, verbose=0) > .5)
y_test_bin_pred_before = np.float32(model_bin.predict(X_test_bin, verbose=0) > .5)
print('\nBefore Training:')
print('Training Data Classification Report:\n', classification_report(y_train_bin, y_train_bin_pred_before))
print('Test Data Classification Report:\n', classification_report(y_test_bin, y_test_bin_pred_before))
print('Training Data Confusion Matrix:\n', confusion_matrix(y_train_bin, y_train_bin_pred_before))
print('Test Data Confusion Matrix:\n', confusion_matrix(y_test_bin, y_test_bin_pred_before))
model_bin.fit(X_train_bin, y_train_bin, batch_size=batch_size, epochs=num_of_epochs, verbose=0)
y_train_bin_pred_after = np.float32(model_bin.predict(X_train_bin, verbose=0) > .5)
y_test_bin_pred_after = np.float32(model_bin.predict(X_test_bin, verbose=0) > .5)
print('\nAfter Training:')
print('Training Data Classification Report:\n', classification_report(y_train_bin, y_train_bin_pred_after))
print('Test Data Classification Report:\n', classification_report(y_test_bin, y_test_bin_pred_after))
print('Training Data Confusion Matrix:\n', confusion_matrix(y_train_bin, y_train_bin_pred_after))
print('Test Data Confusion Matrix:\n', confusion_matrix(y_test_bin, y_test_bin_pred_after))

Model: "model_NetworkType.Binary"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 in (InputLayer)                [(None, 32, 3)]      0           []                               
                                                                                                  
 bn (BatchNormalization)        (None, 32, 3)        12          ['in[0][0]']                     
                                                                                                  
 conv1 (Conv1D)                 (None, 25, 32)       800         ['bn[0][0]']                     
                                                                                                  
 conv2 (Conv1D)                 (None, 25, 32)       800         ['bn[0][0]']                     
                                                                           

2022-12-22 15:07:06.755965: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-22 15:07:06.865777: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-22 15:07:06.906856: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-22 15:07:06.906873: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.



Before Training:
Training Data Classification Report:
               precision    recall  f1-score   support

         0.0       0.58      0.69      0.63     24603
         1.0       0.62      0.50      0.55     24549

    accuracy                           0.60     49152
   macro avg       0.60      0.60      0.59     49152
weighted avg       0.60      0.60      0.59     49152

Test Data Classification Report:
               precision    recall  f1-score   support

         0.0       0.58      0.70      0.63      6117
         1.0       0.63      0.50      0.56      6171

    accuracy                           0.60     12288
   macro avg       0.60      0.60      0.60     12288
weighted avg       0.60      0.60      0.60     12288

Training Data Confusion Matrix:
 [[17084  7519]
 [12260 12289]]
Test Data Confusion Matrix:
 [[4258 1859]
 [3058 3113]]


2022-12-22 15:07:27.732023: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-22 15:07:28.037840: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-22 15:07:28.089691: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-22 15:07:28.089742: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-22 15:07:28.387137: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-22 15:07:28.406751: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-22 15:07:28.534309: I tensorflow/core/grappler/optimizers/cust


After Training:
Training Data Classification Report:
               precision    recall  f1-score   support

         0.0       0.96      0.99      0.97     24603
         1.0       0.99      0.96      0.97     24549

    accuracy                           0.97     49152
   macro avg       0.97      0.97      0.97     49152
weighted avg       0.97      0.97      0.97     49152

Test Data Classification Report:
               precision    recall  f1-score   support

         0.0       0.96      0.99      0.97      6117
         1.0       0.99      0.95      0.97      6171

    accuracy                           0.97     12288
   macro avg       0.97      0.97      0.97     12288
weighted avg       0.97      0.97      0.97     12288

Training Data Confusion Matrix:
 [[24278   325]
 [  985 23564]]
Test Data Confusion Matrix:
 [[6031   86]
 [ 279 5892]]


- Confusion matrices for training and test cases are around 50-50 before training (pure luck)
- It is more than 95% accurate after training which is very impressive
- An output lower than 0.5 means that the network predicts vers=0
- An output greater than 0.5 means that the network predicts vers=1

In [6]:
model_multi = get_model(NetworkType.Multi, X_train_multi)
y_train_multi_argmax = np.argmax(y_train_multi, axis=1)
y_test_multi_argmax = np.argmax(y_test_multi, axis=1)
y_train_multi_pred_before = np.argmax(model_multi.predict(X_train_multi, verbose=0), axis=1)
y_test_multi_pred_before = np.argmax(model_multi.predict(X_test_multi, verbose=0), axis=1)
print('\nBefore Training:')
print('Training Data Classification Report:\n', classification_report(y_train_multi_argmax, y_train_multi_pred_before))
print('Test Data Classification Report:\n', classification_report(y_test_multi_argmax, y_test_multi_pred_before))
print('Training Data Confusion Matrix:\n', confusion_matrix(y_train_multi_argmax, y_train_multi_pred_before))
print('Test Data Confusion Matrix:\n', confusion_matrix(y_test_multi_argmax, y_test_multi_pred_before))
model_multi.fit(X_train_multi, y_train_multi, batch_size=batch_size, epochs=num_of_epochs, verbose=0)
y_train_multi_pred_after = np.argmax(model_multi.predict(X_train_multi, verbose=0), axis=1)
y_test_multi_pred_after = np.argmax(model_multi.predict(X_test_multi, verbose=0), axis=1)
print('\nAfter Training:')
print('Training Data Classification Report:\n', classification_report(y_train_multi_argmax, y_train_multi_pred_after))
print('Test Data Classification Report:\n', classification_report(y_test_multi_argmax, y_test_multi_pred_after))
print('Training Data Confusion Matrix:\n', confusion_matrix(y_train_multi_argmax, y_train_multi_pred_after))
print('Test Data Confusion Matrix:\n', confusion_matrix(y_test_multi_argmax, y_test_multi_pred_after))

Model: "model_NetworkType.Multi"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 in (InputLayer)                [(None, 32, 3)]      0           []                               
                                                                                                  
 bn (BatchNormalization)        (None, 32, 3)        12          ['in[0][0]']                     
                                                                                                  
 conv1 (Conv1D)                 (None, 25, 32)       800         ['bn[0][0]']                     
                                                                                                  
 conv2 (Conv1D)                 (None, 25, 32)       800         ['bn[0][0]']                     
                                                                            

2022-12-22 15:09:03.679744: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-22 15:09:03.796471: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-22 15:09:03.846763: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-22 15:09:03.846783: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.



Before Training:
Training Data Classification Report:
               precision    recall  f1-score   support

           0       0.26      0.89      0.41      6144
           1       0.32      0.01      0.03      6144
           2       0.34      0.03      0.06      6144
           3       0.49      0.23      0.31      6144

    accuracy                           0.29     24576
   macro avg       0.35      0.29      0.20     24576
weighted avg       0.35      0.29      0.20     24576

Test Data Classification Report:
               precision    recall  f1-score   support

           0       0.25      0.89      0.39      1461
           1       0.28      0.01      0.02      1576
           2       0.35      0.03      0.06      1551
           3       0.48      0.21      0.29      1556

    accuracy                           0.28      6144
   macro avg       0.34      0.29      0.19      6144
weighted avg       0.34      0.28      0.19      6144

Training Data Confusion Matrix:
 [[5498 

2022-12-22 15:09:14.697278: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-22 15:09:14.980299: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-22 15:09:15.027711: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-22 15:09:15.027728: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-22 15:09:15.401483: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-22 15:09:15.428178: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-22 15:09:15.563172: I tensorflow/core/grappler/optimizers/cust


After Training:
Training Data Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.83      0.79      6144
           1       0.58      0.59      0.58      6144
           2       0.64      0.64      0.64      6144
           3       0.91      0.79      0.85      6144

    accuracy                           0.71     24576
   macro avg       0.72      0.71      0.72     24576
weighted avg       0.72      0.71      0.72     24576

Test Data Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.82      0.78      1461
           1       0.58      0.58      0.58      1576
           2       0.63      0.64      0.63      1551
           3       0.90      0.78      0.83      1556

    accuracy                           0.70      6144
   macro avg       0.71      0.70      0.71      6144
weighted avg       0.71      0.70      0.70      6144

Training Data Confusion Matrix:
 [[5090 1

- Multi-class case is also trained well but the results are not as impressive as the binary case
- argmax picks the highest vote and it is picked as the predicted class

Previous example: [/examples/time_series/cases/17.ipynb](https://github.com/serhatsoyer/py4ML/blob/main/examples/time_series/cases/17.ipynb)  
Next example: [/examples/toy_datasets.ipynb](https://github.com/serhatsoyer/py4ML/blob/main/examples/toy_datasets.ipynb)